In [14]:
import sqlite3
import pandas as pd
import datetime

# パラメータ
hall_name = "EXA"
model_name = "ジャグラー"

today = datetime.date.today()
end_date = datetime.date(2025, 7, 1)
start_date = datetime.date(2024, 1, 1)


# Table name 取得
DB_PATH = r"C:\python\dataOnline\anaslo_02\db\anaslo_02.db"
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")

cursor.execute(
    "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + hall_name + "%",)
)
results = cursor.fetchall()

query = """
    -- 出玉データにホール名と機種名を結合して取得
    SELECT
        r.*, 
        h.name AS hall_name,
        m.name AS model_name
    FROM results r
    JOIN halls h ON r.hall_id = h.hall_id
    JOIN models m ON r.model_id = m.model_id
    WHERE h.name = ?
    AND r.date BETWEEN ? AND ?
    """

params = [hall_name, start_date, end_date]
if model_name:
    query += " AND m.name LIKE ?"
    params.append(f"%{model_name}%")  # 部分一致にする

query += " ORDER BY r.date DESC, r.unit_no ASC"

df = pd.read_sql_query(query, conn, params=params)

df

,result_id,hall_id,model_id,unit_no,date,game,BB,RB,medals,hall_name,model_name


In [ ]:

# パラメータ
DB_PATH = r"C:\python\dataOnline\anaslo_02\db\anaslo_02.db"
store_name = "EXA"
start_date = pd.to_datetime("2024-01-01")
end_date = pd.to_datetime("2025-12-31")

# データベース接続
conn = sqlite3.connect(DB_PATH)

# 対象店舗の日付データを取得
query = """
SELECT DISTINCT date
FROM slot_data
WHERE store_name = ?
"""
df = pd.read_sql_query(query, conn, params=(store_name,))
conn.close()

# データ型を日付型に変換
df["date"] = pd.to_datetime(df["date"])

# 欠損日付の確認
full_dates = pd.date_range(start=start_date, end=end_date)
missing_dates = full_dates.difference(df["date"])

# 結果出力
print(f"✅ {store_name} のデータ欠損日数: {len(missing_dates)}")
print(missing_dates)
